In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.functional import one_hot
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os
import numpy as np

from dataloader import train_dataloader, val_dataloader, SubjectDataset
from model import BRNN, OneDConvNet
from torch.utils.data import Dataset, DataLoader

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'mps'
input_size = 6
sequence_length = 335413
num_layers = 2
hidden_size = 125
num_classes = 4
learning_rate = 0.0001
batch_size = 128
num_epochs = 20

In [9]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

def train_step(X, y, model, optimizer, criterion):

    y_pred = model(X)
    predicted_classes = torch.argmax(y_pred.detach(), dim=1)

    loss = criterion(y_pred, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    corrects = accuracy_fn(y, y_pred.argmax(dim=1))

    return loss.item(), corrects

def val_step(X, y, model, criterion):

    with torch.no_grad():
        
        y_pred = model(X)
        predicted_classes = torch.argmax(y_pred.detach(), dim=1)
        loss = criterion(y_pred, y)
        corrects = accuracy_fn(y, y_pred.argmax(dim=1))

    return loss.item(), corrects, predicted_classes.detach().cpu().numpy()

In [10]:
best_val_loss = float("inf")
stats = {
  "acc_x": {
    "min": -39.13261,
    "max": 39.26
  },
  "acc_y": {
    "min": -38.92137,
    "max": 39.49
  },
  "acc_z": {
    "min": -31.50025,
    "max": 38.1886
  },
  "gyro_x": {
    "min": -11.62605,
    "max": 10.72668
  },
  "gyro_y": {
    "min": -12.19817,
    "max": 10.93212
  },
  "gyro_z": {
    "min": -6.345545,
    "max": 8.093803
  }
}
min = np.array([v["min"] for k, v in stats.items()])
max = np.array([v["max"] for k, v in stats.items()])

min = torch.from_numpy(min).float()
min = torch.unsqueeze((torch.unsqueeze(min, 0)), -1)
min = min.to(device)
max = torch.from_numpy(max).float().to(device)
max = torch.unsqueeze((torch.unsqueeze(max, 0)), -1)
max = max.to(device)

In [11]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

In [12]:
model = OneDConvNet(input_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=1, verbose=True)

In [13]:
for epoch in range(num_epochs):
    # Train for "n" number of iterations
    running_loss = 0.
    running_acc = 0.
    for iteration, (X, y) in enumerate(train_dataloader):

        X = X.float().to(device)
        # Normalize
        X = (X - min) / (max - min)

        y = y.view(X.size(0)).to(device)

        loss, corrects = train_step(X, y, model, optimizer, criterion)

        # Running metrics
        running_loss = running_loss + loss 
        running_acc = running_acc + corrects

        

    train_loss = running_loss / len(train_dataloader)
    train_acc = running_acc / len(train_dataloader)

    # Validate
    running_val_loss = 0.
    running_val_acc = 0.
    for step, (X, y) in enumerate(val_dataloader):

        X = X.float().to(device)
        X = (X - min) / (max - min)

        y = y.view(X.size(0)).to(device)

        loss, corrects, predicted_classes = val_step(X, y, model, criterion)
        # Running metrics
        running_val_loss = running_val_loss + loss
        running_val_acc = running_val_acc + corrects

    val_loss = running_val_loss / len(val_dataloader)
    val_acc = running_val_acc / len(val_dataloader)

    scheduler.step(val_loss)

    if val_loss < best_val_loss:
        # Checkpoint model
        path = "checkpoint_model_run_1.pth"
        print(f"Saving model to {path}")
        torch.save(model.state_dict(), path)
        best_val_loss = val_loss

    print(f"Epoch: {epoch} | train_loss {train_loss} | train_acc: {train_acc} | val_loss: {val_loss} | val_acc: {val_acc}")

Saving model to checkpoint_model_run_1.pth
Epoch: 0 | train_loss 0.2796646486503591 | train_acc: 88.9086238374193 | val_loss: 0.5636443219565112 | val_acc: 78.36601370073892
Epoch: 1 | train_loss 0.11391738437208988 | train_acc: 96.05295738520927 | val_loss: 0.8632429819125896 | val_acc: 70.87262033045977
Epoch 00003: reducing learning rate of group 0 to 5.0000e-05.
Epoch: 2 | train_loss 0.09180841475448581 | train_acc: 96.82803470246964 | val_loss: 0.6867567057582153 | val_acc: 76.3003194273399
Epoch: 3 | train_loss 0.06924691294471763 | train_acc: 97.68180490654206 | val_loss: 0.6627791451568577 | val_acc: 83.64506362889983
Epoch 00005: reducing learning rate of group 0 to 2.5000e-05.
Epoch: 4 | train_loss 0.06357282886343504 | train_acc: 97.84090139509684 | val_loss: 0.7033233455457222 | val_acc: 80.25178956280787
Epoch: 5 | train_loss 0.05335890559473792 | train_acc: 98.21773485823287 | val_loss: 0.7109544112764555 | val_acc: 80.38648783866995
Epoch 00007: reducing learning rate of

In [14]:
model

OneDConvNet(
  (conv1): Conv1d(6, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (norm1): LayerNorm((3840,), eps=1e-05, elementwise_affine=True)
  (pool1): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  (conv2): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (norm2): LayerNorm((3840,), eps=1e-05, elementwise_affine=True)
  (pool2): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  (conv3): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (norm3): LayerNorm((3840,), eps=1e-05, elementwise_affine=True)
  (pool3): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  (conv4): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
  (norm4): LayerNorm((3840,), eps=1e-05, elementwise_affine=True)
  (fc1): Linear(in_features=256, out_features=512, bias=True)
  (dropout5): Dropout(p=0.4, inplace=False)
  (fc2): Linear(in_features=512, out_features=4, bias=True)
)

In [19]:
import pandas as pd
FILENAME_TEMPLATE = "subject_{}_{}__y.csv"

In [22]:
save_dir = "/Users/purushothamanyadav/Documents/NCSU/Spring23/NN/Project/ProjC/terrain-identification/predictions/C3.1"

In [23]:
test_data_path = "./data/TestData/window_3"
split_ids = ["009_01", "010_01", "011_01", "012_01"]
batch_size = 128
model.eval()
for id in split_ids:

    test_dataset_1sec = SubjectDataset(
        test_data_path, 
        [id]
    )
    test_dataloader_1sec = DataLoader(test_dataset_1sec, batch_size=batch_size, shuffle=False)
    

    
    

    output = []

    for (X_1sec, y_1sec) in test_dataloader_1sec:

        X_1sec = X_1sec.float().to(device)
        X_1sec = (X_1sec - min) / (max - min)

       

        y_pred = model(X_1sec)
        

       
        predicted_classes = torch.argmax(y_pred, dim=1).detach().cpu().numpy()

        output.append(predicted_classes)

    _output = np.concatenate(output, axis=0)

    df = pd.DataFrame({"label": _output})

    subject_id, session_id = id.split("_")

    filename = FILENAME_TEMPLATE.format(subject_id, session_id)
    df.to_csv(os.path.join(save_dir, filename), header=False, index=False)